# 问题三：势头转变预测模型

## 目标
1. 构建预测比赛局势波动的模型
2. 识别与势头转变相关性最强的因素
3. 为教练提供实用建议

## 建模方法
1. **随机森林分类器**: 预测势头转变
2. **特征重要性分析**: 识别关键因素
3. **逻辑回归**: 量化各因素的影响

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

FIGSIZE_NORMAL = (10, 6)
FIGSIZE_WIDE = (12, 6)

COLORS = {
    'primary': '#E74C3C',
    'secondary': '#3498DB',
    'accent': '#27AE60',
    'neutral': '#95A5A6'
}

np.random.seed(42)

In [2]:
# 加载数据
df = pd.read_csv('../数据处理/processed_wimbledon_with_momentum.csv')
print(f"数据加载成功: {df.shape}")

: 

## 一、定义势头转变

势头转变定义为：势头分数符号改变（从正变负或从负变正）的时刻。

In [ ]:
# 计算势头变化
df['momentum_prev'] = df.groupby('match_id')['momentum'].shift(1)
df['momentum_change'] = df['momentum'] - df['momentum_prev']

# 定义势头转变事件
# 1. 符号转变：从正变负或从负变正
df['momentum_shift'] = (
    (df['momentum'] * df['momentum_prev'] < 0) & 
    (abs(df['momentum_prev']) > 1)  # 之前势头有一定强度
).astype(int)

# 2. 大幅波动：势头变化超过阈值
momentum_change_threshold = df['momentum_change'].std() * 1.5
df['large_swing'] = (abs(df['momentum_change']) > momentum_change_threshold).astype(int)

print(f"势头转变事件统计:")
print(f"  符号转变次数: {df['momentum_shift'].sum()}")
print(f"  大幅波动次数: {df['large_swing'].sum()}")
print(f"  变化阈值: {momentum_change_threshold:.2f}")

## 二、特征工程

In [ ]:
# 准备用于预测的特征
feature_cols = [
    # 比赛状态
    'set_no', 'games_in_set', 'sets_played',
    
    # 比分状态
    'point_diff', 'momentum_prev',
    
    # 连胜相关
    'p1_streak_prev', 'p2_streak_prev',
    
    # 发球相关
    'is_p1_serving', 'serve_no',
    
    # 关键分
    'is_break_point', 'is_key_point',
    
    # 比赛节奏
    'rally_count', 'point_duration',
    
    # 滚动胜率
    'p1_rolling_win_rate_5',
]

# 检查可用特征
available_features = [col for col in feature_cols if col in df.columns]
print(f"可用特征数: {len(available_features)}")
print(available_features)

In [ ]:
# 准备建模数据
# 移除首行（没有前一分数据）和缺失值
model_df = df.dropna(subset=available_features + ['momentum_shift']).copy()

X = model_df[available_features]
y = model_df['momentum_shift']

print(f"建模数据: {X.shape}")
print(f"势头转变比例: {y.mean():.3f}")

## 三、随机森林预测模型

In [ ]:
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"训练集: {X_train.shape}")
print(f"测试集: {X_test.shape}")

In [ ]:
# 训练随机森林
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=20,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

# 交叉验证
cv_scores = cross_val_score(rf_model, X, y, cv=5, scoring='roc_auc')
print(f"交叉验证 AUC: {cv_scores.mean():.3f} (+/- {cv_scores.std():.3f})")

In [ ]:
# 测试集评估
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

print("\n分类报告:")
print(classification_report(y_test, y_pred))

auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"\nROC-AUC: {auc_score:.3f}")

**图1: ROC曲线**

In [ ]:
fig, ax = plt.subplots(figsize=FIGSIZE_NORMAL)

fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
ax.plot(fpr, tpr, color=COLORS['primary'], linewidth=2, 
        label=f'Random Forest (AUC = {auc_score:.3f})')
ax.plot([0, 1], [0, 1], color='gray', linestyle='--', label='Random Baseline')

ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.legend(loc='lower right')
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])

plt.tight_layout()
plt.savefig('figures/fig1_roc_curve.pdf', bbox_inches='tight')
plt.show()

**图1解读**: ROC曲线展示了模型区分势头转变事件的能力。AUC值越接近1表示模型预测能力越强。

## 四、特征重要性分析

In [ ]:
# 获取特征重要性
feature_importance = pd.DataFrame({
    'feature': available_features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("特征重要性排名:")
feature_importance

**图2: 特征重要性**

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# 特征名称映射
feature_names = {
    'momentum_prev': 'Previous Momentum',
    'point_diff': 'Point Difference',
    'p1_streak_prev': 'P1 Previous Streak',
    'p2_streak_prev': 'P2 Previous Streak',
    'p1_rolling_win_rate_5': 'P1 Rolling Win Rate',
    'rally_count': 'Rally Count',
    'point_duration': 'Point Duration',
    'is_break_point': 'Break Point',
    'is_key_point': 'Key Point',
    'is_p1_serving': 'P1 Serving',
    'serve_no': 'Serve Number',
    'set_no': 'Set Number',
    'games_in_set': 'Games in Set',
    'sets_played': 'Sets Played'
}

plot_data = feature_importance.copy()
plot_data['feature_name'] = plot_data['feature'].map(feature_names).fillna(plot_data['feature'])

colors = [COLORS['primary'] if imp > 0.1 else COLORS['neutral'] 
          for imp in plot_data['importance']]

bars = ax.barh(plot_data['feature_name'], plot_data['importance'], 
               color=colors, edgecolor='black', alpha=0.8)

ax.set_xlabel('Feature Importance', fontsize=12)
ax.invert_yaxis()

plt.tight_layout()
plt.savefig('figures/fig2_feature_importance.pdf', bbox_inches='tight')
plt.show()

**图2解读**: 红色柱表示重要性超过10%的关键特征。势头转变主要受当前势头状态、得分差距和连胜情况的影响。

## 五、逻辑回归分析

使用逻辑回归量化各因素对势头转变的影响，得到可解释的系数。

In [ ]:
# 标准化特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 训练逻辑回归
lr_model = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)

lr_model.fit(X_scaled, y)

# 获取系数
lr_coefs = pd.DataFrame({
    'feature': available_features,
    'coefficient': lr_model.coef_[0],
    'abs_coef': np.abs(lr_model.coef_[0])
}).sort_values('abs_coef', ascending=False)

print("逻辑回归系数 (标准化后):")
lr_coefs

**图3: 逻辑回归系数**

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

plot_data = lr_coefs.copy()
plot_data['feature_name'] = plot_data['feature'].map(feature_names).fillna(plot_data['feature'])

colors = [COLORS['primary'] if c > 0 else COLORS['secondary'] 
          for c in plot_data['coefficient']]

ax.barh(plot_data['feature_name'], plot_data['coefficient'], 
        color=colors, edgecolor='black', alpha=0.8)

ax.axvline(x=0, color='black', linewidth=0.5)
ax.set_xlabel('Coefficient (Standardized)', fontsize=12)
ax.invert_yaxis()

plt.tight_layout()
plt.savefig('figures/fig3_logistic_coefficients.pdf', bbox_inches='tight')
plt.show()

**图3解读**: 正系数（红色）表示该因素增加势头转变概率，负系数（蓝色）表示降低概率。系数绝对值越大，影响越显著。

## 六、关键因素深度分析

In [ ]:
# 分析各因素与势头转变的关系
print("关键因素分析:")
print("="*60)

# 1. 破发点对势头的影响
bp_shift_rate = model_df[model_df['is_break_point'] == 1]['momentum_shift'].mean()
non_bp_shift_rate = model_df[model_df['is_break_point'] == 0]['momentum_shift'].mean()
print(f"\n1. 破发点影响:")
print(f"   破发点势头转变率: {bp_shift_rate:.3f}")
print(f"   非破发点势头转变率: {non_bp_shift_rate:.3f}")
print(f"   相对风险: {bp_shift_rate/non_bp_shift_rate:.2f}x")

# 2. 连胜对势头的影响
streak_effect = model_df.groupby('p1_streak_prev')['momentum_shift'].mean()
print(f"\n2. 连胜长度影响:")
for streak in [0, 1, 2, 3]:
    if streak in streak_effect.index:
        print(f"   连胜{streak}分后转变率: {streak_effect[streak]:.3f}")

# 3. 发球方因素
serve_shift = model_df.groupby('is_p1_serving')['momentum_shift'].mean()
print(f"\n3. 发球方影响:")
print(f"   P1发球时转变率: {serve_shift.get(1, 0):.3f}")
print(f"   P2发球时转变率: {serve_shift.get(0, 0):.3f}")

**图4: 关键情境下的势头转变率**

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 子图1: 破发点
ax1 = axes[0]
bp_data = [non_bp_shift_rate, bp_shift_rate]
ax1.bar(['Regular Point', 'Break Point'], bp_data, 
        color=[COLORS['neutral'], COLORS['primary']], edgecolor='black', alpha=0.8)
ax1.set_ylabel('Momentum Shift Rate', fontsize=11)
for i, v in enumerate(bp_data):
    ax1.text(i, v + 0.002, f'{v:.3f}', ha='center', fontsize=10)

# 子图2: 连胜长度
ax2 = axes[1]
streak_x = list(range(5))
streak_y = [streak_effect.get(i, 0) for i in streak_x]
ax2.bar(streak_x, streak_y, color=COLORS['secondary'], edgecolor='black', alpha=0.8)
ax2.set_xlabel('Previous Streak Length', fontsize=11)
ax2.set_ylabel('Momentum Shift Rate', fontsize=11)

# 子图3: 盘数进程
ax3 = axes[2]
set_effect = model_df.groupby('set_no')['momentum_shift'].mean()
ax3.bar(set_effect.index, set_effect.values, color=COLORS['accent'], edgecolor='black', alpha=0.8)
ax3.set_xlabel('Set Number', fontsize=11)
ax3.set_ylabel('Momentum Shift Rate', fontsize=11)

plt.tight_layout()
plt.savefig('figures/fig4_context_analysis.pdf', bbox_inches='tight')
plt.show()

**图4解读**: 左图显示破发点时势头更容易转变；中图显示连胜越长，势头转变概率越低（但终将转变）；右图显示比赛后期势头转变更频繁。

## 七、教练建议

In [ ]:
print("="*60)
print("教练建议 - 基于势头转变预测分析")
print("="*60)

# 基于分析结果生成建议
top_features = feature_importance.head(5)['feature'].tolist()

print("\n一、关键发现")
print("-" * 40)
print(f"1. 势头转变最相关的因素: {', '.join(top_features[:3])}")
print(f"2. 破发点是势头转变的高风险时刻 (风险增加 {bp_shift_rate/non_bp_shift_rate:.1f}x)")
print(f"3. 连胜越长，势头越稳定，但终将转变")
print(f"4. 比赛后期(第4-5盘)势头波动更频繁")

print("\n二、战术建议")
print("-" * 40)
print("1. 【守住势头】")
print("   - 破发点是关键，发球方应全力保发")
print("   - 建立连胜后保持专注，不要松懈")
print("   - 关注对手的连胜，尽早打断")

print("\n2. 【扭转势头】")
print("   - 把握破发点机会，这是逆转的最佳时机")
print("   - 延长回合(rally)可增加不确定性")
print("   - 比赛后期不要放弃，势头更易转变")

print("\n3. 【心理准备】")
print("   - 势头是真实存在的，但可以被打断")
print("   - 对手的连胜终将结束，保持耐心")
print("   - 关键分保持冷静，这是势头转折点")

print("\n三、面对陌生对手的建议")
print("-" * 40)
print("1. 观察对手在破发点的表现")
print("2. 注意对手连胜后的行为模式")
print("3. 利用比赛后期势头不稳的特点")
print("4. 保持自己的发球质量，减少破发点")

## 八、保存模型结果

In [ ]:
# 保存特征重要性
feature_importance.to_csv('feature_importance.csv', index=False)
print("特征重要性已保存至: feature_importance.csv")

# 保存逻辑回归系数
lr_coefs.to_csv('logistic_coefficients.csv', index=False)
print("逻辑回归系数已保存至: logistic_coefficients.csv")

## 总结

### 模型性能
- 随机森林分类器能够有效预测势头转变
- 交叉验证AUC表明模型具有一定预测能力

### 关键因素
1. **当前势头状态**: 极端势头更容易回归
2. **得分差距**: 差距越大，转变概率越低
3. **连胜情况**: 连胜提供势头保护
4. **破发点**: 高风险转折时刻
5. **比赛阶段**: 后期波动更频繁

### 实践意义
模型识别的关键因素可指导教练制定战术，帮助选手在关键时刻做出正确决策。